In [ ]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [ ]:
url = 'https://editorial.rottentomatoes.com/guide/140-essential-action-movies-to-watch-now/'